# Using Batteries & Curtailment for Grid Flexibility

This notebook demonstrates how to use flexibility assets—specifically residential batteries and PV curtailment—to alleviate stress on a local electricity grid. The increasing adoption of photovoltaics (PV), heat pumps (HP), and electric vehicles (EVs) can lead to new grid challenges, such as cable overloads and voltage violations, especially during times of high solar generation.

Instead of relying solely on expensive traditional grid reinforcement (i.e., replacing cables), we can use smart technologies to manage these new loads and generation sources. This basic example demonstrates a potential solution to our challenge, though it’s far from perfect. Feel free to unleash your creativity and develop smarter, more effective solutions for your own projects!

### Our Process:
1.  **Analyze the Baseline:** We will select a substation and analyze its performance over a full year to identify any grid failures without any interventions.
2.  **Deploy Batteries:** We'll implement a smart battery strategy where:
    - Customers in weak parts of the grid get stricter export limits.
    - Batteries are optimally sized for each customer to absorb excess solar energy.
    - Batteries discharge to increase customer self-consumption.
3.  **Apply Curtailment:** As a final step, we will apply PV curtailment to handle any rare, extreme events that the battery cannot fully mitigate.
4.  **Compare Results:** We will compare the "before" and "after" scenarios to quantify the improvement.


## 1. Setup: Loading Data and Libraries

First, we load all necessary libraries and datasets. This includes the grid topology, yearly load profiles for consumption, PV, EVs, and HPs, and the costs for grid reinforcement.


In [ ]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
from src.visualization import (
    visualize_network_topology,
    visualize_grid_improvement,
    visualize_network_with_failures
)
from src.functions import (
    build_and_simplify_network,
    find_failures_with_yearly_profile,
    suggest_grid_reinforcement,
    print_analysis_results,
    update_and_save_parquet
)

print("Libraries imported successfully.")

# Loading and Preparing the Data ---
data_path = "../blob-account-edh/challenge-data/"

# Load network topology data
file_path = data_path + "250903_all_stations_anon.csv"
df_full = pd.read_csv(file_path, sep=";")
print("Successfully loaded network data.")

# Load and index all yearly profiles (15-min intervals)
profiles_path = data_path + "data_parquet/"
df_consumption = pd.read_parquet(profiles_path + "base_consumption.parquet").set_index('timestamp')
df_pv = pd.read_parquet(profiles_path + "pv_profiles.parquet").set_index('timestamp')
df_ev = pd.read_parquet(profiles_path + "ev_profiles.parquet").set_index('timestamp')
df_hp = pd.read_parquet(profiles_path + "hp_profiles.parquet").set_index('timestamp')

# Data Preparation and Timestamp Mapping
profiles = {"Consumption": df_consumption, "PV": df_pv, "EV": df_ev, "HP": df_hp}
num_periods = len(df_consumption.index)
datetime_index = pd.date_range(start='2050-01-01', periods=num_periods, freq='15min')
for df in profiles.values():
    df.index = datetime_index
print(f"\nCreated new datetime index from {datetime_index.min()} to {datetime_index.max()}")
print("Successfully loaded and indexed all profiles data.")

# Load reinforcement costs for comparison
df_reinforcement = pd.read_csv(data_path + "190923_Einheitskosten_Invest.csv")
print("Successfully loaded grid reinforcement costs.")

# Calculate the Net Load
# Net Load = (Consumption + EV + HP) - PV Generation
# Note: PV generation is positive, so we subtract it. A negative net load means the customer is exporting power.
NOMINAL_VOLTAGE = 400.0
df_net_load = df_consumption.add(df_ev, fill_value=0).add(df_hp, fill_value=0).subtract(df_pv, fill_value=0)
#df_net_load = df_consumption.subtract(df_pv, fill_value=0)
print("\nSuccessfully calculated the combined net load profile for all customers.")

## 2. Baseline Analysis: Identifying Grid Problems

Now, let's analyze the grid in its initial state. We'll use an interactive dropdown to select a substation. This analysis will simulate the power flow for every 15-minute interval over an entire year to find any cable overloads or voltage violations.

**To follow this example, please select station `Station_1` from the dropdown.** This station is known to have issues due to high PV penetration, making it a perfect case study.


In [ ]:
# Get a list of all unique stations for the dropdown
all_stations = sorted(df_full['station'].unique())

# Create the dropdown and output widgets
station_dropdown = widgets.Dropdown(
    options=all_stations,
    description='Select Station:',
    value='station_1', # Default to a station with known issues
    disabled=False,
)
output_area = widgets.Output()

# This dictionary will store the results of our chosen station for later use
analysis_store = {}

def run_baseline_analysis(selected_station):
    """Runs the initial grid analysis and stores the results."""
    with output_area:
        clear_output(wait=True)
        print(f"--- Running full baseline analysis for station: '{selected_station}' ---")
        
        df_one_station = df_full[df_full['station'] == selected_station].copy()
        
        # Build Network
        G, consumer_props, roots = build_and_simplify_network(df_one_station)
        
        # Run Analysis
        dynamic_results = find_failures_with_yearly_profile(
            graph=G,
            net_profile_df=df_net_load,
            consumer_props=consumer_props,
            root_node_ids=roots,
            nominal_voltage=NOMINAL_VOLTAGE
        )
        
        # Store results for later use in the notebook
        analysis_store['station_id'] = selected_station
        analysis_store['graph'] = G
        analysis_store['consumer_props'] = consumer_props
        analysis_store['root_node_ids'] = roots 
        analysis_store['initial_results'] = dynamic_results
        
        print("\n--- 1. Initial Network Topology ---")
        if len(dynamic_results['link_failures'])==0:
            visualize_network_topology(graph=G, root_node_ids=roots, optimize_space=True)
        else:
            visualize_network_with_failures(
                graph=G, 
                root_node_ids=roots, 
                link_failures=dynamic_results['link_failures'], 
                fuse_failures=dynamic_results["fuse_failures"], 
                optimize_space=True,
                station_name= selected_station
            )
        
        print("\n" + "="*50)
        print_analysis_results("2. RESULTS: Baseline Yearly Profile Analysis", dynamic_results)
        print("="*50)
        print("\nAnalysis complete. The results and network graph are now stored.")
        print("Scroll down to the next section to design and apply our flexibility solution.")


def on_station_change(change):
    run_baseline_analysis(change['new'])

# Link the function and display the UI
station_dropdown.observe(on_station_change, names='value')
display(station_dropdown, output_area)

# Run the analysis for the initial default value
if station_dropdown.value:
    run_baseline_analysis(station_dropdown.value)

### Baseline Results Interpretation

As seen above for station `Station_1`, the analysis identifies multiple **link failures**. These are primarily due to `REVERSE_POWER_FLOW`, which occurs when many households with PV panels export large amounts of solar power back to the grid simultaneously, overloading the local cables.

Our goal is to solve these failures without physically upgrading the cables.

## 3. Solution Design: Battery and Curtailment Logic

We'll now define the functions that model our flexibility solutions.

### Our Strategy
1.  **Dynamic Export Limits:** Instead of a single export limit for everyone, we'll assign a stricter limit (`1.5 kW`) to customers located on grid branches that experienced failures. Customers on stronger parts of the grid get a more generous limit (`5.0 kW`). This targets the solution where it's needed most.
2.  **Optimal Battery Sizing:** For each customer with PV, we'll calculate the smallest battery needed to absorb most of their excess solar energy, respecting their dynamic export limit.
3.  **Smart Battery Control:** The battery will:
    - **Charge** using excess solar power that would have otherwise breached the export limit.
    - **Discharge** to cover the home's own evening consumption, reducing grid imports and increasing self-sufficiency.
4.  **PV Curtailment:** A final "safety net" to clip any power peaks that even the battery cannot handle (e.g., if the battery is already full).



In [ ]:
# Identify Customers with PV Export
customer_peak_generation_kw = -df_net_load.min()
customer_peak_generation_kw = customer_peak_generation_kw[customer_peak_generation_kw > 0]
print(f"Identified {len(customer_peak_generation_kw)} of {len(df_net_load.columns)} customers with PV export across the entire dataset.")

In [ ]:
# Battery Class Definition
class SimpleBattery:
    """A simple battery model that simulates energy storage with constraints."""
    def __init__(self, capacity_kwh, max_power_kw, efficiency=0.9, initial_soc_percent=5.0):
        self.capacity_kwh = float(capacity_kwh)
        self.max_power_kw = float(max_power_kw)
        self.efficiency = float(efficiency)
        self.soc_kwh = self.capacity_kwh * (initial_soc_percent / 100.0)

    def charge(self, power_kw, duration_hours):
        """Charges the battery, returning the actual power used after constraints."""
        power_to_charge = min(power_kw, self.max_power_kw)
        available_capacity_kwh = self.capacity_kwh - self.soc_kwh
        max_energy_in_kwh = available_capacity_kwh / self.efficiency
        max_power_for_duration = max_energy_in_kwh / duration_hours
        actual_power_in = min(power_to_charge, max_power_for_duration)
        energy_added_kwh = actual_power_in * duration_hours * self.efficiency
        self.soc_kwh += energy_added_kwh
        return actual_power_in

    def discharge(self, power_kw, duration_hours):
        """Discharges the battery, returning the actual power supplied after constraints."""
        power_to_discharge = min(power_kw, self.max_power_kw)
        max_power_for_duration = self.soc_kwh / duration_hours
        actual_power_out = min(power_to_discharge, max_power_for_duration)
        energy_removed_kwh = actual_power_out * duration_hours
        self.soc_kwh -= energy_removed_kwh
        return actual_power_out

# Battery Sizing and Scheduling Logic
def calculate_optimal_battery_size(net_load_profile, target_max_export_kw=1.0, capacity_buffer_multiplier=1.2):
    """Calculates required battery power and capacity based on a customer's export profile."""
    duration_hours = 0.25 # 15-minute intervals
    daily_export_energy = net_load_profile.groupby(np.arange(len(net_load_profile)) // 96).apply(lambda day: day[day < 0].sum()) * duration_hours
    
    if not (daily_export_energy < 0).any():
        return (5.0, 10.0) # Default size for non-exporting PV customer

    worst_day_index = daily_export_energy.idxmin()
    worst_day_profile = net_load_profile.iloc[worst_day_index*96 : (worst_day_index+1)*96]
    
    peak_generation_kw = abs(worst_day_profile.min())
    required_power_kw = max(0, peak_generation_kw - target_max_export_kw)
    
    excess_profile = worst_day_profile[worst_day_profile < -target_max_export_kw]
    energy_to_store_kwh = abs(excess_profile.sum() * duration_hours) - (len(excess_profile) * target_max_export_kw * duration_hours)
    
    required_capacity_kwh = energy_to_store_kwh * capacity_buffer_multiplier

    #final_power = max(35.0, required_power_kw)
    #final_capacity = max(40.0, required_capacity_kwh)
    final_power = required_power_kw
    final_capacity = required_capacity_kwh
    return (final_power, final_capacity*3)

def create_battery_schedule(net_load_profile, battery, target_max_export_kw=0.0, min_soc_kwh_reserve=0.0):
    """
    Creates a battery charge/discharge schedule, reserving a minimum SoC for the morning.
    """
    duration_hours = 0.25
    battery_charge_kw, battery_discharge_kw = [], []
    soc_kwh_history = []
    num_timesteps_per_day = 96

    discharge_start_hour = 19
    discharge_end_hour = 4.5
    grid_discharge_timestep_start = discharge_start_hour * 4
    grid_discharge_timestep_end_exclusive = discharge_end_hour * 4

    for i, load_kw in enumerate(net_load_profile):
        charge_for_step, discharge_for_step = 0, 0
        current_timestep_of_day = i % num_timesteps_per_day

        is_grid_discharge_time = (current_timestep_of_day >= grid_discharge_timestep_start) or \
                                 (current_timestep_of_day < grid_discharge_timestep_end_exclusive)

        # Implement the SoC Reserve Logic
        if is_grid_discharge_time:
            # Calculate how much energy is available to discharge above our reserve.
            available_energy_kwh = battery.soc_kwh - min_soc_kwh_reserve

            if available_energy_kwh > 0:
                # We have energy to spare. Discharge it.
                # Note: The battery.discharge method should respect its own max power limits.
                #power_to_discharge_kw = available_energy_kwh / duration_hours
                power_to_discharge_kw = 10.
                discharge_for_step = battery.discharge(power_to_discharge_kw, duration_hours)
        else:
            # Standard operation for daytime (07:00 to 18:59).
            if load_kw < -target_max_export_kw:
                # Excess PV generation: charge the battery.
                power_to_absorb = abs(load_kw) - target_max_export_kw
                charge_for_step = battery.charge(power_to_absorb, duration_hours)
            elif load_kw > 0:
                # Local consumption: discharge the battery to meet this load.
                # This will naturally use the reserve if needed.
                power_to_supply = load_kw
                discharge_for_step = battery.discharge(power_to_supply, duration_hours)

        battery_charge_kw.append(charge_for_step)
        battery_discharge_kw.append(discharge_for_step)
        soc_kwh_history.append(battery.soc_kwh)

    s_charge = pd.Series(battery_charge_kw, index=net_load_profile.index)
    s_discharge = pd.Series(battery_discharge_kw, index=net_load_profile.index)
    s_soc_kwh = pd.Series(soc_kwh_history, index=net_load_profile.index)

    return s_charge, s_discharge, s_soc_kwh


# Dynamic Export Limit Logic
def create_dynamic_export_limits(graph, link_failures_list, consumer_props, root_node_ids, default_limit_kw=20.0, strict_limit_kw=15.0):
    """
    Creates customer-specific export limits based on network topology and failures,
    considering multiple transformer/source nodes.

    Args:
        graph (nx.Graph): The networkx graph of the distribution network.
        link_failures_list (list of dicts): A list of failed links.
        consumer_props (dict): A dictionary of consumer properties.
        root_node_ids (list or set): An iterable of node IDs representing the multiple
                                     transformers or main sources of power.
        default_limit_kw (float): The default export limit for unconstrained customers.
        strict_limit_kw (float): The export limit for customers downstream of a fault.

    Returns:
        dict: A dictionary mapping each customer ID to its calculated export limit in kW.
    """
    print("--- Creating dynamic, location-based export limits for a multi-transformer network ---")
    
    # Create a temporary graph to avoid modifying the original.
    temp_graph = graph.copy()
    
    # Define and add a "super root" that will be the parent of all actual transformer roots.
    # This provides a single, common reference point for distance calculations.
    super_root_id = 'super_root_node' # Use a name that won't conflict with existing nodes
    temp_graph.add_node(super_root_id)
    
    # Connect the super root to all actual transformer roots.
    for root_id in root_node_ids:
        if root_id in temp_graph:
            temp_graph.add_edge(super_root_id, root_id)
        else:
            print(f"Warning: Root node '{root_id}' not found in the graph. Skipping.")

    customer_export_limits = {customer: default_limit_kw for customer in consumer_props.keys()}
    constrained_customers = set()

    if link_failures_list:
        failures_df = pd.DataFrame(link_failures_list)
        for _, row in failures_df.iterrows():
            link_start, link_end = row['link']
            
            # Ensure the failed link's nodes are in our temporary graph before proceeding
            if not temp_graph.has_node(link_start) or not temp_graph.has_node(link_end):
                print(f"Warning: Nodes for failed link ({link_start}, {link_end}) not in graph. Skipping fault.")
                continue

            try:
                # This part now uses the temporary graph and the single super_root_id
                dist_start = nx.shortest_path_length(temp_graph, source=super_root_id, target=link_start)
                dist_end = nx.shortest_path_length(temp_graph, source=super_root_id, target=link_end)
                
                # The logic to determine the downstream node remains the same.
                # The node with the greater distance from the super_root is downstream.
                downstream_node = link_end if dist_end > dist_start else link_start
                
                # The DFS tree is also built on the temporary graph to correctly trace
                # the subgraph that is downstream of the fault.
                subgraph_nodes = nx.dfs_tree(temp_graph, source=downstream_node).nodes()
                
                for node in subgraph_nodes:
                    if node in consumer_props:
                        constrained_customers.add(node)
            except nx.NetworkXNoPath:
                # This might happen if a failed link disconnects a part of the graph
                # from ALL roots. The logic should handle this gracefully.
                print(f"Warning: No path from super_root to nodes of link ({link_start}, {link_end}).")
                continue
                
    for customer in constrained_customers:
        customer_export_limits[customer] = strict_limit_kw
        
    print(f"Identified {len(constrained_customers)} customers on constrained lines requiring strict limits ({strict_limit_kw} kW).")
    return customer_export_limits

print("\nBattery and Control Logic functions defined successfully.")

## 4. Scenario 1: Deploying Batteries with Dynamic Limits

Now we apply our strategy. We will use the results from the baseline analysis of `TS_1011` to generate dynamic export limits and then simulate the addition of optimally-sized batteries for all customers with PV panels in this substation.



In [ ]:
# You might need this import if it's not already in your script
import time

# Generate the Dynamic Export Limits based on original failures ---
G = analysis_store['graph']
initial_results = analysis_store['initial_results']
consumer_props = analysis_store['consumer_props']
root_node_ids = analysis_store['root_node_ids']

customer_export_limits = create_dynamic_export_limits(
    graph=G,
    link_failures_list=initial_results['link_failures'],
    consumer_props=consumer_props,
    root_node_ids=root_node_ids,
    default_limit_kw=20.0, # Generous limit for customers on strong grid sections
    strict_limit_kw=10.   # Strict limit for customers on weak grid sections
)

# Apply Optimal Sizing and create final load profiles
print("\nApplying optimal battery sizing and simulating yearly performance...")

# DataFrames to store results
df_battery_in = pd.DataFrame(0, index=df_net_load.index, columns=df_net_load.columns, dtype=float)
df_battery_out = pd.DataFrame(0, index=df_net_load.index, columns=df_net_load.columns, dtype=float)
df_battery_soc_kwh = pd.DataFrame(0, index=df_net_load.index, columns=df_net_load.columns, dtype=float)


start_time = time.time()
# We only need to simulate for customers connected to the analyzed station
customers_in_station = list(consumer_props.keys())
pv_customers_in_station = customer_peak_generation_kw.index.intersection(customers_in_station)
print(f"Simulating batteries for {len(pv_customers_in_station)} PV customers in station {analysis_store['station_id']}...")

batteries = {}

for customer_id in pv_customers_in_station:
    if customer_id not in df_net_load.columns: continue

    customer_net_load = df_net_load[customer_id]
    target_kw = customer_export_limits.get(customer_id, 20.)

    power_kw, capacity_kwh = calculate_optimal_battery_size(
                                customer_net_load, 
                                target_max_export_kw=0.0,
                                capacity_buffer_multiplier=3.0
                            )
    
    # storing the battery specs for cost calculation later
    batteries[customer_id] = (power_kw, capacity_kwh)
    
    customer_battery = SimpleBattery(capacity_kwh, power_kw)

    # Unpack all three return values from the updated function
    charge_profile, discharge_profile, soc_kwh_profile = create_battery_schedule(
        customer_net_load, 
        customer_battery, 
        target_max_export_kw=target_kw
    )

    # Store the results in our DataFrames
    df_battery_in[customer_id] = charge_profile
    df_battery_out[customer_id] = discharge_profile
    # Store the new SoC profile
    df_battery_soc_kwh[customer_id] = soc_kwh_profile

print(f"Battery simulation completed in {time.time() - start_time:.2f} seconds.")

# store in a file the battery specs
battery_specs_path = f"./results/battery_specs_station_{analysis_store['station_id']}.csv"
battery_specs_df = pd.DataFrame.from_dict(batteries, orient='index', columns=['power_kw', 'capacity_kwh'])
battery_specs_df.index.name = 'customer_id'
battery_specs_df.to_csv(battery_specs_path)


In [ ]:

# Calculate the New Net Load and Re-run the Analysis
# This part remains unchanged as SoC doesn't directly affect the net load calculation
df_net_load_with_batteries = df_net_load.add(df_battery_in, fill_value=0).subtract(df_battery_out, fill_value=0)

print("\nRe-running analysis with battery-modified load profiles...")
results_with_batteries = find_failures_with_yearly_profile(
    graph=G,
    net_profile_df=df_net_load_with_batteries,
    consumer_props=consumer_props,
    root_node_ids=root_node_ids,
    nominal_voltage=NOMINAL_VOLTAGE
)

# Display Comparison
# This part remains unchanged
print("\n\n" + "="*60)
print("--- SCENARIO 1 REPORT: COMPARISON BEFORE vs. AFTER BATTERIES ---")
print("="*60)
print_analysis_results("Original Failures (No Interventions)", initial_results)
print_analysis_results("After Adding Batteries", results_with_batteries)

### Scenario 1 Results

The comparison shows some improvement! The addition of batteries has resolved **some of the grid failures**. The peak reverse power flow has been significantly reduced, bringing the grid back within its operational limits.

This demonstrates that a targeted, intelligent deployment of batteries can be a highly effective alternative to traditional grid reinforcement.


## 5. Visualization: How a Single Battery Performs

Aggregate results are great, but it's more intuitive to see how a single battery operates on a sunny day. Let's pick a customer from a problematic area and visualize their power profile before and after the battery installation.

The plot below shows:
-   **Original Net Load (Blue Dashed):** High export (large negative values) during midday.
-   **Battery Power (Red Bars):** The battery charges (positive red bars) to absorb the excess solar power. It discharges (negative red bars) in the evening to power the home.
-   **Final Net Load (Green):** The new profile as seen by the grid. The midday export is successfully capped at the target limit.
-   **Battery SoC (Purple):** The battery's state of charge, showing it filling up during the day and emptying at night.


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

def create_customer_plot(customer_id, start_day, num_days=7):
    """
    Creates a plot showing how a battery mitigates grid issues for one customer.
    """
    target_kw = customer_export_limits.get(customer_id, 20.)
    full_net_load_profile = df_net_load[customer_id]
    power_kw, capacity_kwh = calculate_optimal_battery_size(full_net_load_profile, target_max_export_kw=0.0, capacity_buffer_multiplier=3.0)
    print(f"Battery capacity = {capacity_kwh}")
    print(f"Battery power = {power_kw}")
    sim_battery = SimpleBattery(capacity_kwh, power_kw, initial_soc_percent=5.0)
    s_charge, s_discharge, s_soc_kwh = create_battery_schedule(
        full_net_load_profile,
        sim_battery,
        target_max_export_kw=0.1
    )
    s_battery_power = s_discharge - s_charge
    s_final_net_load = full_net_load_profile - s_battery_power
    s_soc_percent = (s_soc_kwh / capacity_kwh) * 100
    start_idx = start_day * 96
    end_idx = (start_day + num_days) * 96
    original_net_slice = full_net_load_profile.iloc[start_idx:end_idx]
    final_net_slice = s_final_net_load.iloc[start_idx:end_idx]
    battery_power_slice = s_battery_power.iloc[start_idx:end_idx]
    soc_series_slice = s_soc_percent.iloc[start_idx:end_idx]
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=original_net_slice.index, y=original_net_slice, name='Original Net Load (kW)', line=dict(color='royalblue', dash='dash')), secondary_y=False)
    fig.add_trace(go.Scatter(x=final_net_slice.index, y=final_net_slice, name='Final Net Load (kW)', line=dict(color='green', width=3)), secondary_y=False)
    fig.add_trace(go.Scatter(x=battery_power_slice.index, y=battery_power_slice, name='Battery Power (kW)', marker_color='crimson', mode='lines', fill='tozeroy'), secondary_y=False)
    fig.add_trace(go.Scatter(x=soc_series_slice.index, y=soc_series_slice, name='Battery SoC (%)', line=dict(color='purple')), secondary_y=True)
    fig.update_layout(title_text=f'Battery Operation for Customer {customer_id} (Export Limit: {target_kw} kW)', legend_title_text='Metric', xaxis_title='Timestamp')
    fig.update_yaxes(title_text="<b>Power (kW)</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>State of Charge (%)</b>", range=[0, 100.5], secondary_y=True)
    return fig


def interactive_customer_dashboard():
    """
    Creates and displays the interactive dashboard with a dropdown and a plot.
    """
    customer_dropdown = widgets.Dropdown(
        options=list(df_net_load.columns), # Use list() for safety
        value=df_net_load.columns[10],
        description='Customer:',
        disabled=False,
        style={'description_width': 'initial'}, # Ensures long descriptions aren't cut off
        layout={'width': 'max-content'} # Adjust width to content
    )
    
    plot_output = widgets.Output()

    def on_customer_change(change):
        # Use dictionary key access ['new'] which works for both cases
        new_customer_id = change['new']
        
        # Recalculate the 'worst day' for the selected customer
        worst_day_index = df_net_load[new_customer_id].idxmin().dayofyear - 1
        
        with plot_output:
            plot_output.clear_output(wait=True)
            print(f"Generating plot for {new_customer_id} on their worst export day (Day {worst_day_index+1})...")
            fig = create_customer_plot(
                customer_id=new_customer_id, 
                start_day=worst_day_index, 
                num_days=7
            )
            fig.show()

    customer_dropdown.observe(on_customer_change, names='value')

    print("Select a customer from the dropdown to visualize their battery performance.")
    display(customer_dropdown, plot_output)
    
    # Manually trigger the callback for the initial plot using a dictionary
    on_customer_change({'new': customer_dropdown.value})


# Run the dashboard
interactive_customer_dashboard()

## 6. Scenario 2: Adding Curtailment as a Final Polish

In our case, the batteries solved some of the problems. However, in more extreme scenarios, a few violations might remain. Curtailment acts as a final backstop. It involves instructing the PV inverter to momentarily reduce its output to prevent grid issues.

Here, we define the curtailment logic and run it on our battery-corrected profiles. This will confirm that the grid remains stable even if we add this final layer of control.


In [ ]:

# Curtailment Logic Function
def apply_curtailment(net_load_after_battery_df, customer_export_limits):
    """Applies PV curtailment to cap exports at the specified limits."""
    print("\n--- Applying Final PV Curtailment as a Safety Measure ---")
    curtailed_net_load = net_load_after_battery_df.copy()
    duration_hours = 0.25 
    curtailed_energy_kwh = pd.DataFrame(0.0, index=net_load_after_battery_df.index, columns=net_load_after_battery_df.columns)

    for customer_id, limit_kw in customer_export_limits.items():
        if customer_id not in curtailed_net_load.columns: continue
        
        export_limit = -abs(limit_kw) + 8
        customer_profile = curtailed_net_load[customer_id]
        violation_mask = customer_profile < export_limit
        
        if violation_mask.any():
            curtailed_power = export_limit - customer_profile[violation_mask]
            curtailed_energy_kwh.loc[violation_mask, customer_id] = curtailed_power * duration_hours
            curtailed_net_load.loc[violation_mask, customer_id] = export_limit
            
    total_curtailed = curtailed_energy_kwh.sum().sum()
    print(f"Curtailment applied. Total energy curtailed: {total_curtailed:,.2f} kWh/year.")
    return curtailed_net_load, curtailed_energy_kwh

# Run the Definitive Simulation with Batteries + Curtailment
df_net_load_final, df_curtailed_energy = apply_curtailment(
    df_net_load_with_batteries,
    customer_export_limits
)

definitive_results = find_failures_with_yearly_profile(
    graph=G,
    net_profile_df=df_net_load_final,
    consumer_props=consumer_props,
    root_node_ids=root_node_ids,
    nominal_voltage=NOMINAL_VOLTAGE
)

# Display Definitive Results
print("\n\n" + "="*60)
print("--- DEFINITIVE REPORT: BATTERIES + PV CURTAILMENT ---")
print("="*60)
print_analysis_results("Definitive Scenario (Batteries + Curtailment)", definitive_results)

total_curtailed_by_customer = df_curtailed_energy.sum().sort_values(ascending=False)
print("\n--- Top Customers by Total Curtailed Energy (kWh/year) ---")
display(total_curtailed_by_customer[total_curtailed_by_customer > 0].head(10))

In [ ]:
# Make sure you have these DataFrames available from running your script
# df_net_load_with_batteries  <- The profile after battery simulation
# df_net_load_final           <- The profile after curtailment is applied
# customer_export_limits      <- The dictionary of limits

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

def create_curtailment_plot(customer_id, start_day, num_days=2):
    """
    Creates a plot visualizing the effect of PV curtailment for a single customer.
    """
    # Get data for the specific customer
    profile_before_curtail = df_net_load_with_batteries[customer_id]
    profile_after_curtail = df_net_load_final[customer_id]
    export_limit_kw = customer_export_limits.get(customer_id, 1.5)
    
    # Calculate the amount of power curtailed
    # Curtailed power is the difference between the 'before' and 'after' profiles.
    # It will be > 0 only when curtailment is active.
    power_curtailed = profile_before_curtail - profile_after_curtail
    
    # Get the relevant data slices for the plotting period
    start_idx = start_day * 96
    end_idx = (start_day + num_days) * 96
    
    before_slice = profile_before_curtail.iloc[start_idx:end_idx]
    after_slice = profile_after_curtail.iloc[start_idx:end_idx]
    curtailed_slice = power_curtailed.iloc[start_idx:end_idx]

    # Create the Plotly Figure
    fig = go.Figure()

    # Trace 1: Net Load after battery (the 'before' state)
    fig.add_trace(go.Scatter(
        x=before_slice.index, 
        y=before_slice, 
        name='Net Load after Battery (kW)', 
        line=dict(color='deepskyblue', dash='dash')
    ))

    # Trace 2: Final Net Load after curtailment (the 'after' state)
    fig.add_trace(go.Scatter(
        x=after_slice.index, 
        y=after_slice, 
        name='Final Net Load (kW)', 
        line=dict(color='darkgreen', width=3)
    ))
    
    # Trace 3: Curtailed Power (visualized as a filled area)
    fig.add_trace(go.Scatter(
        x=curtailed_slice.index,
        y=curtailed_slice,
        name='Power Curtailed (kW)',
        fill='tozeroy',
        mode='lines',
        line=dict(width=0, color='rgba(220, 20, 60, 0.5)'), # Transparent crimson
        fillcolor='rgba(220, 20, 60, 0.5)'
    ))

    # Add a horizontal line for the export limit
    fig.add_hline(
        y=-abs(export_limit_kw), 
        line_dash="dot",
        annotation_text=f"Export Limit: {-abs(export_limit_kw):.2f} kW", 
        annotation_position="bottom right",
        line_color="crimson"
    )

    fig.update_layout(
        title_text=f'PV Curtailment for Customer {customer_id}',
        xaxis_title='Timestamp',
        yaxis_title='Power (kW)',
        legend_title_text='Metric'
    )
    
    return fig

In [ ]:
def interactive_curtailment_dashboard():
    """
    Creates and displays an interactive dashboard for visualizing curtailment.
    """
    # To make the dropdown more useful, let's populate it with customers
    # who actually had some energy curtailed.
    customers_with_curtailment = total_curtailed_by_customer[total_curtailed_by_customer > 0].index.tolist()
    
    if not customers_with_curtailment:
        print("No customers had any energy curtailed. Dashboard not needed.")
        return

    # Create the Dropdown widget
    customer_dropdown = widgets.Dropdown(
        options=customers_with_curtailment,
        value=customers_with_curtailment[0], # Default to the most curtailed customer
        description='Customer:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': 'max-content'}
    )
    
    # Create the Output widget to hold the plot
    plot_output = widgets.Output()

    # Define the callback function that updates the plot
    def on_customer_change(change):
        new_customer_id = change['new']
        
        # We want to find the day where the PRE-curtailment profile had its worst export
        worst_day_index = df_net_load_with_batteries[new_customer_id].idxmin().dayofyear - 1
        
        with plot_output:
            plot_output.clear_output(wait=True)
            print(f"Generating curtailment plot for {new_customer_id} on their worst export day (Day {worst_day_index+1})...")
            
            # Call our new plotting function
            fig = create_curtailment_plot(
                customer_id=new_customer_id, 
                start_day=worst_day_index, 
                num_days=2
            )
            fig.show()

    # Link the dropdown's 'value' to the callback function
    customer_dropdown.observe(on_customer_change, names='value')

    # Display the widgets
    print("Select a customer from the dropdown to visualize their PV curtailment.")
    display(customer_dropdown, plot_output)
    
    # Manually trigger the callback for the initial plot
    on_customer_change({'new': customer_dropdown.value})

# --- Run the new dashboard ---
# Place this code in a new cell AFTER you have calculated df_net_load_final, 
# df_net_load_with_batteries, and total_curtailed_by_customer.

interactive_curtailment_dashboard()

### Definitive Results and Conclusion

The final analysis confirms that the combination of batteries and a minimal amount of curtailment creates a robust grid. The amount of energy curtailed is negligible, showing that the batteries did almost all of the heavy lifting.

In summary, this notebook has demonstrated:
1.  **A Problem:** High PV penetration can cause reverse power flow issues on local grids.
2.  **An Intelligent Solution:** By using dynamic export limits based on grid topology, we can strategically deploy flexibility assets.
3.  **Effective Tools:** Optimally-sized residential batteries are highly effective at absorbing excess solar generation for later use, solving the vast majority of grid problems.
4.  **A Reliable Backstop:** PV curtailment can act as a final, low-impact measure to guarantee grid stability.

This data-driven approach allows grid operators to integrate more renewable energy sources efficiently and cost-effectively, deferring or avoiding expensive physical upgrades.

## 7. Save Flexibility Profiles to Parquet Files

Finally, we will save the generated battery and curtailment profiles to Parquet files. This allows us to reuse these results later without needing to re-run the entire simulation.

The following code is designed to be **idempotent**:
- If a profile file (e.g., `battery_in_profiles.parquet`) doesn't exist, it will be created with the data from the station we just analyzed.
- If the file **does** exist, it will be loaded, and only the columns corresponding to the customers from our analyzed station will be updated or added. This preserves data from any previous analyses of other stations.


In [ ]:
df_curtailed_energy

In [ ]:
import os # Make sure os is imported

# The path where the profiles will be saved
output_profiles_path = data_path + "data_parquet/"

# The list of customers that were part of the recent analysis
customers_analyzed = list(analysis_store['consumer_props'].keys())

# Define all profiles to be saved in a structured list
profiles_to_save = [
    {
        "df": df_battery_in,
        "filename": "battery_in_profiles.parquet",
        "description": "Battery Charging Profiles (kW)"
    },
    {
        "df": df_battery_out,
        "filename": "battery_out_profiles.parquet",
        "description": "Battery Discharging Profiles (kW)"
    },
    {
        "df": df_battery_soc_kwh,  # <-- ADDED THE NEW SOC PROFILE
        "filename": "battery_soc_profiles.parquet",
        "description": "Battery State of Charge Profiles (kW)"
    },
    {
        "df": df_curtailed_energy,
        "filename": "curtailed_energy_profiles.parquet",
        "description": "Curtailed Energy Profiles (kWh)"
    }
]

print("\n" + "="*50)
print(f"Saving results for {len(customers_analyzed)} customers from station: {analysis_store['station_id']}")
print("="*50 + "\n")

# Loop through the list and save each profile
for profile in profiles_to_save:
    print(f"-> Saving: {profile['description']}...")
    update_and_save_parquet(
        new_data_df=profile['df'],
        file_path=os.path.join(output_profiles_path, profile['filename']),
        customers_to_update=customers_analyzed
    )

print("\n--- All flexibility and state profiles have been saved successfully. ---")

In [ ]:
import pandas as pd
import os

def update_and_save_parquet(new_data_df, file_path, customers_to_update):
    """
    Saves or updates a Parquet file with new profile data for a specific set of customers.
    DEFINITIVE FIX: Handles cases where the existing file has a non-datetime index (e.g., 'V1').
    It merges based on row position and applies the correct DatetimeIndex from the new data.

    Args:
        new_data_df (pd.DataFrame): DataFrame with a proper DatetimeIndex containing the new profile data.
        file_path (str): The full path to the Parquet file to be saved.
        customers_to_update (list): A list of customer IDs whose data should be updated.
    """
    # Filter the new data and ensure it has a proper DatetimeIndex
    relevant_new_data = new_data_df[customers_to_update].copy()
    relevant_new_data.index = pd.to_datetime(relevant_new_data.index)
    
    # Store the correct index for later use
    correct_index = relevant_new_data.index

    if os.path.exists(file_path):
        print(f"File '{os.path.basename(file_path)}' exists. Loading and updating...")
        try:
            existing_df = pd.read_parquet(file_path)

            # Critical Check: Ensure row counts match. If not, we cannot safely merge.
            if len(existing_df) != len(relevant_new_data):
                print(f"  [Warning] Row count mismatch! Existing file has {len(existing_df)} rows, "
                      f"new data has {len(relevant_new_data)}. Overwriting file to prevent data corruption.")
                # Force overwrite by raising an exception to jump to the 'except' block
                raise ValueError("Row count mismatch")

            # Temporarily remove indices to prepare for a position-based merge.
            # This is the key step to handle the 'V1' vs. DatetimeIndex conflict.
            existing_df_vals = existing_df.reset_index(drop=True)
            new_data_vals = relevant_new_data.reset_index(drop=True)
            
            # Identify columns to drop from the existing data
            cols_to_drop = existing_df_vals.columns.intersection(new_data_vals.columns)
            df_for_others = existing_df_vals.drop(columns=cols_to_drop)
            
            # Concatenate based on the temporary RangeIndex (0, 1, 2...)
            final_df_no_index = pd.concat([df_for_others, new_data_vals], axis=1)

            # Restore the correct DatetimeIndex
            final_df = final_df_no_index.set_index(correct_index)
            
            # (Optional but good practice) Sort columns for consistency
            final_df = final_df.reindex(sorted(final_df.columns), axis=1)
            
            print(f"Successfully merged data for {len(customers_to_update)} customers.")

        except Exception as e:
            print(f"  An error occurred during the merge process: {e}. Defaulting to overwrite with new data.")
            final_df = relevant_new_data
    else:
        print(f"File '{os.path.basename(file_path)}' does not exist. Creating new file...")
        final_df = relevant_new_data

    # Save the final DataFrame
    try:
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        final_df.to_parquet(file_path, index=True)
        print(f"Successfully saved data to '{os.path.basename(file_path)}'")
    except Exception as e:
        print(f"  [ERROR] Failed to save file {file_path}: {e}")

In [ ]:
import os
import pandas as pd # Required for checking if a DataFrame is empty

# --- FIX: Ensure the output directory exists before trying to save files ---
# The path where the profiles will be saved
output_profiles_path = "./results/data_parquet/"
os.makedirs(output_profiles_path, exist_ok=True) # This line creates the folder if it doesn't exist.

# --- SETUP: Assuming these variables are defined from your analysis ---
# The list of customers that were part of the recent analysis
customers_analyzed = list(analysis_store['consumer_props'].keys())

# --- UPDATE: Define ALL available profiles to be saved in a structured list ---
# NOTE: This assumes all these DataFrames (df_battery_in, df_net_load, etc.) exist.
profiles_to_save = [
    # Flexibility / Battery Profiles
    {
        "df": df_battery_in,
        "filename": "battery_in_profiles.parquet",
        "description": "Battery Charging Profiles (kW)"
    },
    {
        "df": df_battery_out,
        "filename": "battery_out_profiles.parquet",
        "description": "Battery Discharging Profiles (kW)"
    },
    {
        "df": df_battery_soc_kwh,
        "filename": "battery_soc_profiles.parquet",
        "description": "Battery State of Charge Profiles (kWh)"
    },
    {
        "df": df_curtailed_energy,
        "filename": "curtailed_energy_profiles.parquet",
        "description": "Curtailed Energy Profiles (kWh)"
    },
    # Base Load, Generation, and Net Profiles
    {
        "df": df_consumption,
        "filename": "base_consumption.parquet",
        "description": "Base Consumption Profiles (kW)"
    },
    {
        "df": df_pv,
        "filename": "pv_profiles.parquet",
        "description": "PV Generation Profiles (kW)"
    },
    {
        "df": df_ev,
        "filename": "ev_profiles.parquet",
        "description": "EV Charging Profiles (kW)"
    },
    {
        "df": df_hp,
        "filename": "hp_profiles.parquet",
        "description": "Heat Pump Consumption Profiles (kW)"
    },
    {
        "df": df_net_load,
        "filename": "net_load_profiles.parquet",
        "description": "Net Load Profiles (kW)"
    }
]

# --- Saving Logic (Unchanged from your original code) ---
print("\n" + "="*50)
print(f"Saving results for {len(customers_analyzed)} customers from station: {analysis_store['station_id']}")
print("="*50 + "\n")

# Loop through the list and save each profile
for profile in profiles_to_save:
    # Check if the dataframe to save is not empty or None
    if profile['df'] is not None and not profile['df'].empty:
        print(f"-> Saving: {profile['description']}...")
        # NOTE: This assumes your 'update_and_save_parquet' function is defined elsewhere
        update_and_save_parquet(
            new_data_df=profile['df'],
            file_path=os.path.join(output_profiles_path, profile['filename']),
            customers_to_update=customers_analyzed
        )
    else:
        print(f"-> Skipping: {profile['description']} (DataFrame is empty or None).")


print("\n--- All available profiles have been saved successfully. ---")